In [6]:
from __future__ import print_function
import os
import sys

In [7]:
# Someday this has to go in a config or kernel

#spark_home = '/opt/spark/latest'

#os.environ['SPARK_HOME'] = spark_home
#sys.path.insert(0, spark_home + "/python")
#sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.1-src.zip'))

#from pyspark.context import SparkContext
#from pyspark.context import SparkConf
#from pyspark.sql import SQLContext

In [8]:
#sc = SparkContext()
#sqlContext = SQLContext(sc)

In [9]:
df = sqlContext.read.load("/guoda/data/idigbio-20161119.parquet")

In [10]:
print(df.count())
df.head(10)

158673


[Row(barcodevalue=None, basisofrecord='preservedspecimen', canonicalname='castilleja pallida subsp. septentrionalis (lindl.) h.j. scoggan', catalognumber='82798', class='magnoliopsida', collectioncode='herb', collectionid=None, collector='c. g. pringle', commonname=None, continent='north america', country='united states', countrycode='usa', datecollected=datetime.datetime(1877, 6, 21, 19, 3, 58), family='orobanchaceae', fieldnumber=None, genus='castilleja', group=None, highertaxon='plantae | dicots | orobanchaceae', individualcount=None, infraspecificepithet=None, kingdom='plantae', municipality=None, order='lamiales', phylum='tracheophyta', scientificname='castilleja septentrionalis', specificepithet='septentrionalis', stateprovince='vermont', uuid='0b10740c-3e61-4e45-93ca-29e4b8ca5b15', waterbody=None),
 Row(barcodevalue=None, basisofrecord='preservedspecimen', canonicalname='huperzia selago (l.) bernh.', catalognumber='204078', class='lycopodiopsida', collectioncode='herb', collecti